<h1>Setup</h1>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import seaborn as sns # data visualization library  
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['LANL-Earthquake-Prediction', 'even-more-features']


This Kernel will use this kernel: https://www.kaggle.com/artgor/even-more-features/ features as input features.

In [2]:
%%time
print(os.listdir("../input/even-more-features"))
train_features = pd.read_csv('../input/even-more-features/train_features.csv')
test_features = pd.read_csv('../input/even-more-features/test_features.csv')
y = pd.read_csv('../input/even-more-features/y.csv')

['__notebook__.ipynb', 'submission.csv', '__output__.json', '__results___files', 'custom.css', 'y.csv', 'test_features.csv', '__results__.html', 'train_features.csv']
CPU times: user 2.28 s, sys: 168 ms, total: 2.45 s
Wall time: 2.57 s


<h1>Preprocessing Train and Test Data</h1>

Scaling data with standart scaler

In [3]:
%%time
scaler = StandardScaler()
scaler.fit(train_features)
X_train_scaled = pd.DataFrame(scaler.transform(train_features), columns=train_features.columns)
X_test_scaled = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 160 ms, sys: 176 ms, total: 336 ms
Wall time: 336 ms


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


split data train 80 % and test 20

In [4]:
%%time
x_train, x_valid, y_train, y_valid = train_test_split(X_train_scaled, y, test_size=0.2, random_state=78)

CPU times: user 24 ms, sys: 24 ms, total: 48 ms
Wall time: 49.3 ms


<h1>Dropping highly correlated features</h1>

In [5]:
%%time
corr_matrix = x_train.corr()
#f,ax = plt.subplots(figsize=(18, 18))
#sns.heatmap(corr_matrix, annot=True, linewidths=.5, fmt= '.1f',ax=ax)

CPU times: user 11.6 s, sys: 28 ms, total: 11.7 s
Wall time: 11.7 s


Select upper triangle of correlation matrix

In [6]:
corr_matrix = corr_matrix.abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper

,Hann_window_mean_150,Hann_window_mean_1500,Hann_window_mean_15000,Hann_window_mean_50,Hilbert_mean,abs_max,abs_max_roll_mean_10,abs_max_roll_mean_100,abs_max_roll_mean_1000,abs_max_roll_mean_10000,abs_max_roll_mean_50,abs_max_roll_mean_500,abs_max_roll_std_10,abs_max_roll_std_100,abs_max_roll_std_1000,abs_max_roll_std_10000,abs_max_roll_std_50,abs_max_roll_std_500,abs_mean,abs_percentile_1,abs_percentile_10,abs_percentile_20,abs_percentile_25,abs_percentile_30,abs_percentile_40,abs_percentile_5,abs_percentile_50,abs_percentile_60,abs_percentile_70,abs_percentile_75,abs_percentile_80,abs_percentile_90,abs_percentile_95,abs_percentile_99,abs_std,abs_trend,autocorrelation_10,autocorrelation_100,autocorrelation_1000,autocorrelation_10000,...,range_-3000_-2000,range_-4000_-3000,range_0_1000,range_1000_2000,range_2000_3000,range_3000_4000,range_minf_m4000,range_p4000_pinf,skew,spkt_welch_density_1,spkt_welch_density_10,spkt_welch_density_100,spkt_welch_density_5,spkt_welch_density_50,std,std_first_1000,std_first_10000,std_first_50000,std_last_1000,std_last_10000,std_last_50000,std_roll_mean_10,std_roll_mean_100,std_roll_mean_1000,std_roll_mean_10000,std_roll_mean_50,std_roll_mean_500,std_roll_std_10,std_roll_std_100,std_roll_std_1000,std_roll_std_10000,std_roll_std_50,std_roll_std_500,sum,time_rev_asym_stat_1,time_rev_asym_stat_10,time_rev_asym_stat_100,time_rev_asym_stat_5,time_rev_asym_stat_50,trend
Hann_window_mean_150,NaN,0.999999,0.999963,1.000000,0.083330,0.018539,0.020842,0.024626,0.099450,0.720851,0.014454,0.060341,0.013315,0.014517,0.016377,0.017596,0.016204,0.014007,0.147784,NaN,0.248716,0.461795,0.705889,0.408417,0.462260,0.510017,0.609534,0.562829,0.390635,0.369216,0.255472,0.129799,0.065246,0.023095,0.019356,0.000934,0.020053,0.075639,0.000880,0.009395,...,0.013062,0.003078,0.313329,0.019515,0.014761,0.000501,0.010633,0.027147,0.002473,0.010381,0.013742,0.040113,0.024114,0.039858,0.016930,0.012595,0.006930,0.022042,0.021484,0.003849,0.016300,0.016816,0.019521,0.022120,0.006971,0.016685,0.021311,0.016276,0.016973,0.017321,0.018025,0.016981,0.017129,1.000000,0.020822,0.007378,0.007634,0.012756,0.001781,0.012516
Hann_window_mean_1500,NaN,NaN,0.999969,0.999999,0.083441,0.018661,0.020966,0.024752,0.099580,0.720941,0.014584,0.060477,0.013437,0.014640,0.016499,0.017718,0.016327,0.014128,0.147895,NaN,0.248747,0.461832,0.705907,0.408473,0.462313,0.510035,0.609584,0.562845,0.390680,0.369296,0.255532,0.129896,0.065361,0.023204,0.019478,0.000826,0.020050,0.075617,0.000892,0.009392,...,0.013186,0.002935,0.313290,0.019629,0.014883,0.000620,0.010562,0.027311,0.002502,0.010527,0.013877,0.040181,0.024224,0.039990,0.017052,0.012592,0.006941,0.022051,0.021521,0.003932,0.016436,0.016940,0.019659,0.022263,0.007016,0.016821,0.021457,0.016393,0.017094,0.017443,0.018151,0.017101,0.017251,0.999999,0.020767,0.007250,0.007653,0.012637,0.001676,0.012531
Hann_window_mean_15000,NaN,NaN,NaN,0.999963,0.083840,0.019154,0.021460,0.025285,0.100126,0.721307,0.015183,0.061098,0.013967,0.015158,0.017045,0.018262,0.016833,0.014645,0.148309,NaN,0.248987,0.461855,0.705917,0.408588,0.462341,0.510036,0.609555,0.562679,0.390686,0.369374,0.255537,0.130202,0.065729,0.023665,0.020005,0.000012,0.020098,0.075489,0.000811,0.009255,...,0.013861,0.002498,0.313338,0.020183,0.015483,0.001183,0.010531,0.027802,0.002623,0.011115,0.014428,0.040331,0.024682,0.040268,0.017568,0.012362,0.006715,0.021928,0.021536,0.004100,0.017344,0.017461,0.020230,0.022762,0.007143,0.017429,0.022005,0.016903,0.017615,0.017970,0.018710,0.017619,0.017775,0.999963,0.020432,0.006547,0.007932,0.012109,0.000663,0.012656
Hann_window_mean_50,NaN,NaN,NaN,NaN,0.083328,0.018533,0.020836,0.024620,0.099443,0.720846,0.014445,0.060333,0.013307,0.014509,0.016369,0.017590,0.016196,0.014000,0.147782,NaN,0.248712,0.461796,0.705890,0.408420,0.462258,0.510017,0.609534,0.562834,0.390639,0.369217,0.255476,0.129804,0.065248,0.023092,0.019350,0.000939,0.020052,0.075642,0.000875,0.009394,...,0.013055,0.003094,0.313326,0

Find index of feature columns with correlation greater than 0.95

In [7]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

['Hann_window_mean_1500',
 'Hann_window_mean_15000',
 'Hann_window_mean_50',
 'abs_max_roll_mean_10',
 'abs_max_roll_mean_100',
 'abs_max_roll_mean_1000',
 'abs_max_roll_mean_50',
 'abs_max_roll_mean_500',
 'abs_max_roll_std_10',
 'abs_max_roll_std_100',
 'abs_max_roll_std_1000',
 'abs_max_roll_std_10000',
 'abs_max_roll_std_50',
 'abs_max_roll_std_500',
 'abs_mean',
 'abs_percentile_90',
 'abs_percentile_95',
 'abs_percentile_99',
 'abs_std',
 'av_change_abs_roll_std_50',
 'av_change_rate_roll_std_10',
 'av_change_rate_roll_std_100',
 'av_change_rate_roll_std_1000',
 'av_change_rate_roll_std_10000',
 'av_change_rate_roll_std_50',
 'av_change_rate_roll_std_500',
 'ave10',
 'ave_roll_mean_10',
 'ave_roll_mean_100',
 'ave_roll_mean_1000',
 'ave_roll_mean_10000',
 'ave_roll_mean_50',
 'ave_roll_mean_500',
 'ave_roll_std_10',
 'ave_roll_std_100',
 'ave_roll_std_1000',
 'ave_roll_std_10000',
 'ave_roll_std_50',
 'ave_roll_std_500',
 'binned_entropy_50',
 'binned_entropy_60',
 'binned_entrop

Drop columns with correlation greater than 0.95

In [8]:
x_train_corr = x_train.drop(to_drop, axis=1)
x_valid_corr = x_valid.drop(to_drop, axis=1)
X_test_scaled_corr = X_test_scaled.drop(to_drop, axis=1)

In [9]:
print("Training Shape", x_train_corr.shape)
print("Validating Shape", x_valid_corr.shape)
print("Testing Shape", X_test_scaled_corr.shape)

Training Shape (3356, 455)
Validating Shape (839, 455)
Testing Shape (2624, 455)


 <h1>Recursive feature elimination (RFE) with random forest</h1>

In [10]:
%%time
rf = RandomForestRegressor(n_estimators=2)
rfe = RFE(estimator=rf, n_features_to_select=100, step=1, verbose=10)
rfe = rfe.fit(x_train_corr, y_train.values)
print('Chosen best 100 feature by rfe:',x_train_corr.columns[rfe.support_])

Fitting estimator with 455 features.


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting estimator with 454 features.
Fitting estimator with 453 features.
Fitting estimator with 452 features.
Fitting estimator with 451 features.
Fitting estimator with 450 features.
Fitting estimator with 449 features.
Fitting estimator with 448 features.
Fitting estimator with 447 features.
Fitting estimator with 446 features.
Fitting estimator with 445 features.
Fitting estimator with 444 features.
Fitting estimator with 443 features.
Fitting estimator with 442 features.
Fitting estimator with 441 features.
Fitting estimator with 440 features.
Fitting estimator with 439 features.
Fitting estimator with 438 features.
Fitting estimator with 437 features.
Fitting estimator with 436 features.
Fitting estimator with 435 features.
Fitting estimator with 434 features.
Fitting estimator with 433 features.
Fitting estimator with 432 features.
Fitting estimator with 431 features.
Fitting estimator with 430 features.
Fitting estimator with 429 features.
Fitting estimator with 428 features.
F

Fitting estimator with 232 features.
Fitting estimator with 231 features.
Fitting estimator with 230 features.
Fitting estimator with 229 features.
Fitting estimator with 228 features.
Fitting estimator with 227 features.
Fitting estimator with 226 features.
Fitting estimator with 225 features.
Fitting estimator with 224 features.
Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
Fitting estimator with 216 features.
Fitting estimator with 215 features.
Fitting estimator with 214 features.
Fitting estimator with 213 features.
Fitting estimator with 212 features.
Fitting estimator with 211 features.
Fitting estimator with 210 features.
Fitting estimator with 209 features.
Fitting estimator with 208 features.
Fitting estimator with 207 features.
Fitting estimator with 206 features.
F

<h1>Recursive feature elimination with cross validation and random forest regression<h2>

In [11]:
%%time
rf = RandomForestRegressor(n_estimators=2)
rfecv = RFECV(estimator=rf, step=1, cv=3, scoring='neg_mean_absolute_error', verbose=10)   #3-fold cross-validation with mae
rfecv = rfecv.fit(x_train_corr, y_train.values)
print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train_corr.columns[rfecv.support_])

Fitting estimator with 455 features.


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting estimator with 454 features.
Fitting estimator with 453 features.
Fitting estimator with 452 features.
Fitting estimator with 451 features.
Fitting estimator with 450 features.
Fitting estimator with 449 features.
Fitting estimator with 448 features.
Fitting estimator with 447 features.
Fitting estimator with 446 features.
Fitting estimator with 445 features.
Fitting estimator with 444 features.
Fitting estimator with 443 features.
Fitting estimator with 442 features.
Fitting estimator with 441 features.
Fitting estimator with 440 features.
Fitting estimator with 439 features.
Fitting estimator with 438 features.
Fitting estimator with 437 features.
Fitting estimator with 436 features.
Fitting estimator with 435 features.
Fitting estimator with 434 features.
Fitting estimator with 433 features.
Fitting estimator with 432 features.
Fitting estimator with 431 features.
Fitting estimator with 430 features.
Fitting estimator with 429 features.
Fitting estimator with 428 features.
F

Fitting estimator with 232 features.
Fitting estimator with 231 features.
Fitting estimator with 230 features.
Fitting estimator with 229 features.
Fitting estimator with 228 features.
Fitting estimator with 227 features.
Fitting estimator with 226 features.
Fitting estimator with 225 features.
Fitting estimator with 224 features.
Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
Fitting estimator with 216 features.
Fitting estimator with 215 features.
Fitting estimator with 214 features.
Fitting estimator with 213 features.
Fitting estimator with 212 features.
Fitting estimator with 211 features.
Fitting estimator with 210 features.
Fitting estimator with 209 features.
Fitting estimator with 208 features.
Fitting estimator with 207 features.
Fitting estimator with 206 features.
F

Fitting estimator with 455 features.
Fitting estimator with 454 features.
Fitting estimator with 453 features.
Fitting estimator with 452 features.
Fitting estimator with 451 features.
Fitting estimator with 450 features.
Fitting estimator with 449 features.
Fitting estimator with 448 features.
Fitting estimator with 447 features.
Fitting estimator with 446 features.
Fitting estimator with 445 features.
Fitting estimator with 444 features.
Fitting estimator with 443 features.
Fitting estimator with 442 features.
Fitting estimator with 441 features.
Fitting estimator with 440 features.
Fitting estimator with 439 features.
Fitting estimator with 438 features.
Fitting estimator with 437 features.
Fitting estimator with 436 features.
Fitting estimator with 435 features.
Fitting estimator with 434 features.
Fitting estimator with 433 features.
Fitting estimator with 432 features.
Fitting estimator with 431 features.
Fitting estimator with 430 features.
Fitting estimator with 429 features.
F

Fitting estimator with 233 features.
Fitting estimator with 232 features.
Fitting estimator with 231 features.
Fitting estimator with 230 features.
Fitting estimator with 229 features.
Fitting estimator with 228 features.
Fitting estimator with 227 features.
Fitting estimator with 226 features.
Fitting estimator with 225 features.
Fitting estimator with 224 features.
Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
Fitting estimator with 216 features.
Fitting estimator with 215 features.
Fitting estimator with 214 features.
Fitting estimator with 213 features.
Fitting estimator with 212 features.
Fitting estimator with 211 features.
Fitting estimator with 210 features.
Fitting estimator with 209 features.
Fitting estimator with 208 features.
Fitting estimator with 207 features.
F

Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 455 features.
Fitting estimator with 454 features.
Fitting estimator with 453 features.
Fitting estimator with 452 features.
Fitting estimator with 451 features.
Fitting estimator with 450 features.
Fitting estimator with 449 features.
Fitting estimator with 448 features.
Fitting estimator with 447 features.
Fitting estimator with 446 features.
Fitting estimator with 445 features.
Fitting estimator with 444 features.
Fitting estimator with 443 features.
Fitting estimator with 442 features.
Fitting estimator with 441 features.
Fitting estimator with 440 features.
Fitting estimator with 439 features.
Fitting estimator with 438 features.
Fitting estimator with 437 features.
Fitting estimator with 436 features.
Fitting estimator with 435 features.
Fitting estimator with 434 features.
Fitting estimator with 433 features.
Fitting estimator with 432 features.
Fitting

Fitting estimator with 236 features.
Fitting estimator with 235 features.
Fitting estimator with 234 features.
Fitting estimator with 233 features.
Fitting estimator with 232 features.
Fitting estimator with 231 features.
Fitting estimator with 230 features.
Fitting estimator with 229 features.
Fitting estimator with 228 features.
Fitting estimator with 227 features.
Fitting estimator with 226 features.
Fitting estimator with 225 features.
Fitting estimator with 224 features.
Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
Fitting estimator with 216 features.
Fitting estimator with 215 features.
Fitting estimator with 214 features.
Fitting estimator with 213 features.
Fitting estimator with 212 features.
Fitting estimator with 211 features.
Fitting estimator with 210 features.
F

Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 455 features.
Fitting estimator with 454 features.
Fitting estimator with 453 features.
Fitting estimator with 452 features.
Fitting estimator with 451 features.
Fitting estimator with 450 features.
Fitting estimator with 449 features.
Fitting estimator with 448 features.
Fitting estimator with 447 features.
Fitting estimator with 446 features.
Fitting estimator with 445 features.
Fitting estimator with 444 features.
Fitting estimator with 443 features.
Fitting estimator with 442 features.
Fitting estimator with 441 features.
Fitting estimator with 440 features.
Fitting estimator with 439 features.
Fitting estimator w

Fitting estimator with 243 features.
Fitting estimator with 242 features.
Fitting estimator with 241 features.
Fitting estimator with 240 features.
Fitting estimator with 239 features.
Fitting estimator with 238 features.
Fitting estimator with 237 features.
Fitting estimator with 236 features.
Fitting estimator with 235 features.
Fitting estimator with 234 features.
Fitting estimator with 233 features.
Fitting estimator with 232 features.
Fitting estimator with 231 features.
Fitting estimator with 230 features.
Fitting estimator with 229 features.
Fitting estimator with 228 features.
Fitting estimator with 227 features.
Fitting estimator with 226 features.
Fitting estimator with 225 features.
Fitting estimator with 224 features.
Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
F

Filter scaled train and test data with optimal features.

In [12]:
x_train_corr_rfecv = x_train_corr[x_train_corr.columns[rfecv.support_].values]
x_valid_corr_rfecv = x_valid_corr[x_valid_corr.columns[rfecv.support_].values]
X_test_scaled_corr_refcv = X_test_scaled_corr[X_test_scaled_corr.columns[rfecv.support_].values]

In [13]:
print("Training Shape", x_train_corr_rfecv.shape)
print("Validating Shape", x_valid_corr_rfecv.shape)
print("Testing Shape", X_test_scaled_corr_refcv.shape)

Training Shape (3356, 56)
Validating Shape (839, 56)
Testing Shape (2624, 56)


<h1>Cross Validator Functions and Hyper Parameter Optimizations</h1>

In [50]:
#import required packages
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import gc
from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample
#optional but advised
import warnings
warnings.filterwarnings('ignore')

#GLOBAL HYPEROPT PARAMETERS
NUM_EVALS = 1000 #number of hyperopt evaluation rounds
N_FOLDS = 5 #number of cross-validation folds on data in each evaluation round

#LIGHTGBM PARAMETERS
LGBM_MAX_LEAVES = 2**11 #maximum number of leaves per tree for LightGBM
LGBM_MAX_DEPTH = 25 #maximum tree depth for LightGBM
EVAL_METRIC_LGBM_REG = 'mae' #LightGBM regression metric. Note that 'rmse' is more commonly used 
EVAL_METRIC_LGBM_CLASS = 'auc'#LightGBM classification metric

#XGBOOST PARAMETERS
XGB_MAX_LEAVES = 2**12 #maximum number of leaves when using histogram splitting
XGB_MAX_DEPTH = 25 #maximum tree depth for XGBoost
EVAL_METRIC_XGB_REG = 'mae' #XGBoost regression metric
EVAL_METRIC_XGB_CLASS = 'auc' #XGBoost classification metric

#CATBOOST PARAMETERS
CB_MAX_DEPTH = 8 #maximum tree depth in CatBoost
OBJECTIVE_CB_REG = 'MAE' #CatBoost regression metric
OBJECTIVE_CB_CLASS = 'Logloss' #CatBoost classification metric

def quick_hyperopt(data, labels, package='lgbm', num_evals=NUM_EVALS, diagnostic=False):
    
    #==========
    #LightGBM
    #==========
    
    if package=='lgbm':
        
        print('Running {} rounds of LightGBM parameter optimisation:'.format(num_evals))
        #clear space
        gc.collect()
        
        integer_params = ['max_depth',
                         'num_leaves',
                          'max_bin',
                         'min_data_in_leaf',
                         'min_data_in_bin']
        
        def objective(space_params):
            
            #cast integer params from float to int
            for param in integer_params:
                space_params[param] = int(space_params[param])
            
            #extract nested conditional parameters
            if space_params['boosting']['boosting'] == 'goss':
                top_rate = space_params['boosting'].get('top_rate')
                other_rate = space_params['boosting'].get('other_rate')
                #0 <= top_rate + other_rate <= 1
                top_rate = max(top_rate, 0)
                top_rate = min(top_rate, 0.5)
                other_rate = max(other_rate, 0)
                other_rate = min(other_rate, 0.5)
                space_params['top_rate'] = top_rate
                space_params['other_rate'] = other_rate
            
            subsample = space_params['boosting'].get('subsample', 1.0)
            space_params['boosting'] = space_params['boosting']['boosting']
            space_params['subsample'] = subsample
            
            #for classification, set stratified=True and metrics=EVAL_METRIC_LGBM_CLASS
            cv_results = lgb.cv(space_params, train, nfold = N_FOLDS, stratified=False,
                                early_stopping_rounds=100, metrics=EVAL_METRIC_LGBM_REG, seed=42)
            
            best_loss = cv_results['l1-mean'][-1] #'l2-mean' for rmse
            #for classification, comment out the line above and uncomment the line below:
            #best_loss = 1 - cv_results['auc-mean'][-1]
            #if necessary, replace 'auc-mean' with '[your-preferred-metric]-mean'
            return{'loss':best_loss, 'status': STATUS_OK }
        
        train = lgb.Dataset(data, labels)
                
        #integer and string parameters, used with hp.choice()
        boosting_list = [{'boosting': 'gbdt',
                          'subsample': hp.uniform('subsample', 0.5, 1)},
                         {'boosting': 'goss',
                          'subsample': 1.0,
                         'top_rate': hp.uniform('top_rate', 0, 0.5),
                         'other_rate': hp.uniform('other_rate', 0, 0.5)}] #if including 'dart', make sure to set 'n_estimators'
        metric_list = ['MAE', 'RMSE'] 
        #for classification comment out the line above and uncomment the line below
        #modify as required for other classification metrics classification
        #metric_list = ['auc']
        objective_list_reg = ['huber', 'gamma', 'fair', 'tweedie']
        objective_list_class = ['logloss', 'cross_entropy']
        #for classification set objective_list = objective_list_class
        objective_list = objective_list_reg

        space ={'boosting' : hp.choice('boosting', boosting_list),
                'num_leaves' : hp.quniform('num_leaves', 2, LGBM_MAX_LEAVES, 1),
                'max_depth': hp.quniform('max_depth', 2, LGBM_MAX_DEPTH, 1),
                'max_bin': hp.quniform('max_bin', 32, 255, 1),
                'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 256, 1),
                'min_data_in_bin': hp.quniform('min_data_in_bin', 1, 256, 1),
                'lambda_l1' : hp.uniform('lambda_l1', 0, 5),
                'lambda_l2' : hp.uniform('lambda_l2', 0, 5),
                'learning_rate' : hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
                'metric' : hp.choice('metric', metric_list),
                'objective' : hp.choice('objective', objective_list),
                'feature_fraction' : hp.quniform('feature_fraction', 0.5, 1, 0.01),
                'bagging_fraction' : hp.quniform('bagging_fraction', 0.5, 1, 0.01)
            }
        
        #optional: activate GPU for LightGBM
        #follow compilation steps here:
        #https://www.kaggle.com/vinhnguyen/gpu-acceleration-for-lightgbm/
        #then uncomment lines below:
        #space['device'] = 'gpu'
        #space['gpu_platform_id'] = 0,
        #space['gpu_device_id'] =  0

        trials = Trials()
        best = fmin(fn=objective,
                    space=space,
                    algo=tpe.suggest,
                    max_evals=num_evals, 
                    trials=trials)
                
        #fmin() will return the index of values chosen from the lists/arrays in 'space'
        #to obtain actual values, index values are used to subset the original lists/arrays
        best['boosting'] = boosting_list[best['boosting']]['boosting']#nested dict, index twice
        best['metric'] = metric_list[best['metric']]
        best['objective'] = objective_list[best['objective']]
        
        #cast floats of integer params to int
        for param in integer_params:
            best[param] = int(best[param])
            
        print('{' + '\n'.join('{}: {}'.format(k, v) for k, v in best.items()) + '}')
        if diagnostic:
            return(best, trials)
        else:
            return(best)
    
    #==========
    #XGBoost
    #==========
    
    if package=='xgb':
        
        print('Running {} rounds of XGBoost parameter optimisation:'.format(num_evals))
        #clear space
        gc.collect()
        
        integer_params = ['max_depth']
        
        def objective(space_params):
            
            for param in integer_params:
                space_params[param] = int(space_params[param])
                
            #extract multiple nested tree_method conditional parameters
            #libera te tutemet ex inferis
            if space_params['tree_method']['tree_method'] == 'hist':
                max_bin = space_params['tree_method'].get('max_bin')
                space_params['max_bin'] = int(max_bin)
                if space_params['tree_method']['grow_policy']['grow_policy']['grow_policy'] == 'depthwise':
                    grow_policy = space_params['tree_method'].get('grow_policy').get('grow_policy').get('grow_policy')
                    space_params['grow_policy'] = grow_policy
                    space_params['tree_method'] = 'hist'
                else:
                    max_leaves = space_params['tree_method']['grow_policy']['grow_policy'].get('max_leaves')
                    space_params['grow_policy'] = 'lossguide'
                    space_params['max_leaves'] = int(max_leaves)
                    space_params['tree_method'] = 'hist'
            else:
                space_params['tree_method'] = space_params['tree_method'].get('tree_method')
                
            #for classification replace EVAL_METRIC_XGB_REG with EVAL_METRIC_XGB_CLASS
            cv_results = xgb.cv(space_params, train, nfold=N_FOLDS, metrics=[EVAL_METRIC_XGB_REG],
                             early_stopping_rounds=100, stratified=False, seed=42)
            
            best_loss = cv_results['test-mae-mean'].iloc[-1] #or 'test-rmse-mean' if using RMSE
            #for classification, comment out the line above and uncomment the line below:
            #best_loss = 1 - cv_results['test-auc-mean'].iloc[-1]
            #if necessary, replace 'test-auc-mean' with 'test-[your-preferred-metric]-mean'
            return{'loss':best_loss, 'status': STATUS_OK }
        
        train = xgb.DMatrix(data, labels)
        
        #integer and string parameters, used with hp.choice()
        boosting_list = ['gbtree', 'gblinear'] #if including 'dart', make sure to set 'n_estimators'
        metric_list = ['MAE', 'RMSE'] 
        #for classification comment out the line above and uncomment the line below
        #metric_list = ['auc']
        #modify as required for other classification metrics classification
        
        tree_method = [{'tree_method' : 'exact'},
               {'tree_method' : 'approx'},
               {'tree_method' : 'hist',
                'max_bin': hp.quniform('max_bin', 2**3, 2**7, 1),
                'grow_policy' : {'grow_policy': {'grow_policy':'depthwise'},
                                'grow_policy' : {'grow_policy':'lossguide',
                                                  'max_leaves': hp.quniform('max_leaves', 32, XGB_MAX_LEAVES, 1)}}}]
        
        #if using GPU, replace 'exact' with 'gpu_exact' and 'hist' with
        #'gpu_hist' in the nested dictionary above
        
        objective_list_reg = ['reg:linear', 'reg:gamma', 'reg:tweedie']
        objective_list_class = ['reg:logistic', 'binary:logistic']
        #for classification change line below to 'objective_list = objective_list_class'
        objective_list = objective_list_reg
        
        space ={'boosting' : hp.choice('boosting', boosting_list),
                'tree_method' : hp.choice('tree_method', tree_method),
                'max_depth': hp.quniform('max_depth', 2, XGB_MAX_DEPTH, 1),
                'reg_alpha' : hp.uniform('reg_alpha', 0, 5),
                'reg_lambda' : hp.uniform('reg_lambda', 0, 5),
                'min_child_weight' : hp.uniform('min_child_weight', 0, 5),
                'gamma' : hp.uniform('gamma', 0, 5),
                'learning_rate' : hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
                'eval_metric' : hp.choice('eval_metric', metric_list),
                'objective' : hp.choice('objective', objective_list),
                'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1, 0.01),
                'colsample_bynode' : hp.quniform('colsample_bynode', 0.1, 1, 0.01),
                'colsample_bylevel' : hp.quniform('colsample_bylevel', 0.1, 1, 0.01),
                'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
                'nthread' : -1
            }
        
        #optional: activate GPU for XGBoost
        #uncomment line below
        #space['tree_method'] = 'gpu_hist'
        
        trials = Trials()
        best = fmin(fn=objective,
                    space=space,
                    algo=tpe.suggest,
                    max_evals=num_evals, 
                    trials=trials)
        
        best['tree_method'] = tree_method[best['tree_method']]['tree_method']
        best['boosting'] = boosting_list[best['boosting']]
        best['eval_metric'] = metric_list[best['eval_metric']]
        best['objective'] = objective_list[best['objective']]
        
        #cast floats of integer params to int
        for param in integer_params:
            best[param] = int(best[param])
        if 'max_leaves' in best:
            best['max_leaves'] = int(best['max_leaves'])
        if 'max_bin' in best:
            best['max_bin'] = int(best['max_bin'])
        
        print('{' + '\n'.join('{}: {}'.format(k, v) for k, v in best.items()) + '}')
        
        if diagnostic:
            return(best, trials)
        else:
            return(best)
    
    #==========
    #CatBoost
    #==========
    
    if package=='cb':
        
        print('Running {} rounds of CatBoost parameter optimisation:'.format(num_evals))
        
        #clear memory 
        gc.collect()
            
        integer_params = ['depth',
                          #'one_hot_max_size', #for categorical data
                          'min_data_in_leaf',
                          'max_bin']
        
        def objective(space_params):
                        
            #cast integer params from float to int
            for param in integer_params:
                space_params[param] = int(space_params[param])
                
            #extract nested conditional parameters
            if space_params['bootstrap_type']['bootstrap_type'] == 'Bayesian':
                bagging_temp = space_params['bootstrap_type'].get('bagging_temperature')
                space_params['bagging_temperature'] = bagging_temp
                
            #if space_params['grow_policy']['grow_policy'] == 'LossGuide':
            #    max_leaves = space_params['grow_policy'].get('max_leaves')
            #    space_params['max_leaves'] = int(max_leaves)
                
            space_params['bootstrap_type'] = space_params['bootstrap_type']['bootstrap_type']
            #space_params['grow_policy'] = space_params['grow_policy']['grow_policy']
                           
            #random_strength cannot be < 0
            space_params['random_strength'] = max(space_params['random_strength'], 0)
            #fold_len_multiplier cannot be < 1
            space_params['fold_len_multiplier'] = max(space_params['fold_len_multiplier'], 1)
                       
            #for classification set stratified=True
            cv_results = cb.cv(train, space_params, fold_count=N_FOLDS, 
                             early_stopping_rounds=25, stratified=False, partition_random_seed=42)
           
            best_loss = cv_results['test-MAE-mean'].iloc[-1] #'test-RMSE-mean' for RMSE
            #for classification, comment out the line above and uncomment the line below:
            #best_loss = cv_results['test-Logloss-mean'].iloc[-1]
            #if necessary, replace 'test-Logloss-mean' with 'test-[your-preferred-metric]-mean'
            
            return{'loss':best_loss, 'status': STATUS_OK}
        
        train = cb.Pool(data, labels.astype('float32'))
        
        #integer and string parameters, used with hp.choice()
        bootstrap_type = [
                          {'bootstrap_type':'Poisson'}, 
                           {'bootstrap_type':'Bayesian',
                            'bagging_temperature' : hp.loguniform('bagging_temperature', np.log(1), np.log(50))},
                          {'bootstrap_type':'Bernoulli'}] 
        LEB = ['No', 'AnyImprovement', 'Armijo'] #remove 'Armijo' if not using GPU
        #score_function = ['Correlation', 'L2', 'NewtonCorrelation', 'NewtonL2']
        grow_policy = [{'grow_policy':'SymmetricTree'},
                       #{'grow_policy':'Depthwise'},
                       #{'grow_policy':'Lossguide',
                       # 'max_leaves': hp.quniform('max_leaves', 2, 32, 1)}
                      ]
        eval_metric_list_reg = ['MAE', 'RMSE', 'Poisson']
        eval_metric_list_class = ['Logloss', 'AUC', 'F1']
        #for classification change line below to 'eval_metric_list = eval_metric_list_class'
        eval_metric_list = eval_metric_list_reg
                
        space ={'depth': hp.quniform('depth', 2, CB_MAX_DEPTH, 1),
                'max_bin' : hp.quniform('max_bin', 1, 32, 1), #if using CPU just set this to 254
                #'max_bin': 254,
                'l2_leaf_reg' : hp.uniform('l2_leaf_reg', 0, 5),
                'min_data_in_leaf' : hp.quniform('min_data_in_leaf', 1, 50, 1),
                'random_strength' : hp.loguniform('random_strength', np.log(0.005), np.log(5)),
                #'one_hot_max_size' : hp.quniform('one_hot_max_size', 2, 16, 1), #uncomment if using categorical features
                'bootstrap_type' : hp.choice('bootstrap_type', bootstrap_type),
                'learning_rate' : hp.uniform('learning_rate', 0.05, 0.25),
                'eval_metric' : hp.choice('eval_metric', eval_metric_list),
                'objective' : OBJECTIVE_CB_REG,
                #'score_function' : hp.choice('score_function', score_function), #crashes kernel - reason unknown
                'leaf_estimation_backtracking' : hp.choice('leaf_estimation_backtracking', LEB),
                #'grow_policy': hp.choice('grow_policy', grow_policy),
                'colsample_bylevel' : hp.quniform('colsample_bylevel', 0.1, 1, 0.01),# CPU only
                'fold_len_multiplier' : hp.loguniform('fold_len_multiplier', np.log(1.01), np.log(2.5)),
                'od_type' : 'Iter',
                'od_wait' : 25,
                'task_type' : 'GPU',
                'verbose' : 0
            }
        
        #optional: run CatBoost without GPU
        #uncomment line below
        #space['task_type'] = 'CPU'
            
        trials = Trials()
        best = fmin(fn=objective,
                    space=space,
                    algo=tpe.suggest,
                    max_evals=num_evals, 
                    trials=trials)
        
        #unpack nested dicts first
        best['bootstrap_type'] = bootstrap_type[best['bootstrap_type']]['bootstrap_type']
        #best['grow_policy'] = grow_policy[best['grow_policy']]['grow_policy']
        best['eval_metric'] = eval_metric_list[best['eval_metric']]
        
        best['score_function'] = score_function[best['score_function']] 
        #best['leaf_estimation_method'] = LEM[best['leaf_estimation_method']] #CPU only
        best['leaf_estimation_backtracking'] = LEB[best['leaf_estimation_backtracking']]        
        
        #cast floats of integer params to int
        for param in integer_params:
            best[param] = int(best[param])
        if 'max_leaves' in best:
            best['max_leaves'] = int(best['max_leaves'])
        
        print('{' + '\n'.join('{}: {}'.format(k, v) for k, v in best.items()) + '}')
        
        if diagnostic:
            return(best, trials)
        else:
            return(best)
    
    else:
        print('Package not recognised. Please use "lgbm" for LightGBM, "xgb" for XGBoost or "cb" for CatBoost.')     

<h2>Optimize Hyperparameters for XGB</h2>

In [21]:
xgb_params = quick_hyperopt(x_train_corr_rfecv, y_train, 'xgb', 2500)
np.save('xgb_params.npy', xgb_params)

Running 2500 rounds of XGBoost parameter optimisation:
100%|██████████| 2500/2500 [47:42<00:00,  1.84s/it, best loss: 2.0891872]          
{boosting: gbtree
colsample_bylevel: 0.74
colsample_bynode: 0.31
colsample_bytree: 1.0
eval_metric: MAE
gamma: 1.2861053004892011
learning_rate: 0.19987589307498949
max_depth: 4
min_child_weight: 4.992656946548014
objective: reg:linear
reg_alpha: 1.5106408696730107
reg_lambda: 0.021714935285173147
subsample: 0.9500000000000001
tree_method: approx}


<h2>Optimize Hyperparameters for LGB</h2>

In [23]:
lgbm_params = quick_hyperopt(x_train_corr_rfecv, y_train, 'lgbm', 1500)
np.save('lgbm_params.npy', lgbm_params)

Running 1500 rounds of LightGBM parameter optimisation:
100%|██████████| 1500/1500 [22:19<00:00,  1.11s/it, best loss: 2.058066255151868] 
{bagging_fraction: 0.79
boosting: gbdt
feature_fraction: 0.78
lambda_l1: 3.9138829422260972
lambda_l2: 4.0651574498215
learning_rate: 0.10716823366484945
max_bin: 48
max_depth: 23
metric: RMSE
min_data_in_bin: 72
min_data_in_leaf: 180
num_leaves: 714
objective: fair
subsample: 0.6862953597478736}


<h2>Optimize Hyperparameters for Catboost</h2>

In [52]:
#cb_params = quick_hyperopt(x_train_corr_rfecv, y_train, 'cb', 30)
#np.save('cb_params.npy', cb_params)

<h1>Training & Prediction</h1>

Read submission file

In [26]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

<h2>XGB</h2>

In [53]:
xgb_params['eval_metric'] = 'mae'
model = xgb.XGBModel(**xgb_params)
model.fit(x_train_corr_rfecv, y_train, 
          eval_set=[(x_train_corr_rfecv, y_train), (x_valid_corr_rfecv, y_valid)], 
          verbose=True)

test_preds = model.predict(X_test_scaled_corr_refcv)
test_segs = X_test_scaled_corr_refcv.index
submission_xgb = pd.DataFrame(columns=['seg_id', 'time_to_failure'])
submission_xgb.seg_id = test_segs
submission_xgb.time_to_failure = test_preds
submission_xgb.to_csv('submission_xgb.csv', index=False, line_terminator='\n')

[0]	validation_0-mae:4.31092	validation_1-mae:4.1415
[1]	validation_0-mae:3.60703	validation_1-mae:3.47184
[2]	validation_0-mae:3.07145	validation_1-mae:2.97032
[3]	validation_0-mae:2.67368	validation_1-mae:2.61635
[4]	validation_0-mae:2.39812	validation_1-mae:2.37597
[5]	validation_0-mae:2.21353	validation_1-mae:2.23165
[6]	validation_0-mae:2.09747	validation_1-mae:2.14772
[7]	validation_0-mae:2.02604	validation_1-mae:2.09599
[8]	validation_0-mae:1.98029	validation_1-mae:2.06859
[9]	validation_0-mae:1.95017	validation_1-mae:2.06072
[10]	validation_0-mae:1.92694	validation_1-mae:2.06137
[11]	validation_0-mae:1.90899	validation_1-mae:2.06409
[12]	validation_0-mae:1.89019	validation_1-mae:2.06667
[13]	validation_0-mae:1.87888	validation_1-mae:2.06832
[14]	validation_0-mae:1.86728	validation_1-mae:2.06479
[15]	validation_0-mae:1.85645	validation_1-mae:2.07068
[16]	validation_0-mae:1.84272	validation_1-mae:2.07172
[17]	validation_0-mae:1.82937	validation_1-mae:2.07533
[18]	validation_0-mae

In [58]:
submission_xgb

,seg_id,time_to_failure
0,0,2.990885
1,1,4.775519
2,2,5.642636
3,3,8.393229
4,4,7.578168
5,5,2.962801
6,6,9.136414
7,7,4.771952
8,8,4.429375
9,9,2.812222


<h2>LGB</h2>

In [55]:
model = lgb.train(lgbm_params, 
                  lgb.Dataset(x_train_corr_rfecv,y_train), 
                  num_boost_round=20_000, 
                  valid_sets=lgb.Dataset(x_valid_corr_rfecv,y_valid), 
                  early_stopping_rounds=2000, 
                  verbose_eval=True)

test_preds = model.predict(X_test_scaled_corr_refcv)
test_segs = X_test_scaled_corr_refcv.index
submission_lgbm = pd.DataFrame(columns=['seg_id', 'time_to_failure'])
submission_lgbm.seg_id = test_segs
submission_lgbm.time_to_failure = test_preds
submission_lgbm.to_csv('submission_lgbm.csv', index=False, line_terminator='\n')

[1]	valid_0's rmse: 3.38633
Training until validation scores don't improve for 2000 rounds.
[2]	valid_0's rmse: 3.20726
[3]	valid_0's rmse: 3.07259
[4]	valid_0's rmse: 2.98055
[5]	valid_0's rmse: 2.91261
[6]	valid_0's rmse: 2.85949
[7]	valid_0's rmse: 2.81667
[8]	valid_0's rmse: 2.78428
[9]	valid_0's rmse: 2.7598
[10]	valid_0's rmse: 2.74245
[11]	valid_0's rmse: 2.72375
[12]	valid_0's rmse: 2.71081
[13]	valid_0's rmse: 2.7032
[14]	valid_0's rmse: 2.69401
[15]	valid_0's rmse: 2.68677
[16]	valid_0's rmse: 2.68286
[17]	valid_0's rmse: 2.67845
[18]	valid_0's rmse: 2.67536
[19]	valid_0's rmse: 2.67267
[20]	valid_0's rmse: 2.67034
[21]	valid_0's rmse: 2.66861
[22]	valid_0's rmse: 2.66687
[23]	valid_0's rmse: 2.66529
[24]	valid_0's rmse: 2.66667
[25]	valid_0's rmse: 2.66427
[26]	valid_0's rmse: 2.66216
[27]	valid_0's rmse: 2.65952
[28]	valid_0's rmse: 2.66084
[29]	valid_0's rmse: 2.66246
[30]	valid_0's rmse: 2.66357
[31]	valid_0's rmse: 2.6622
[32]	valid_0's rmse: 2.66235
[33]	valid_0's rmse:

[326]	valid_0's rmse: 2.71842
[327]	valid_0's rmse: 2.71792
[328]	valid_0's rmse: 2.71776
[329]	valid_0's rmse: 2.71823
[330]	valid_0's rmse: 2.71751
[331]	valid_0's rmse: 2.71735
[332]	valid_0's rmse: 2.71877
[333]	valid_0's rmse: 2.71916
[334]	valid_0's rmse: 2.71895
[335]	valid_0's rmse: 2.71915
[336]	valid_0's rmse: 2.71847
[337]	valid_0's rmse: 2.7187
[338]	valid_0's rmse: 2.71975
[339]	valid_0's rmse: 2.72038
[340]	valid_0's rmse: 2.7205
[341]	valid_0's rmse: 2.72047
[342]	valid_0's rmse: 2.72122
[343]	valid_0's rmse: 2.72076
[344]	valid_0's rmse: 2.72101
[345]	valid_0's rmse: 2.72162
[346]	valid_0's rmse: 2.72146
[347]	valid_0's rmse: 2.72179
[348]	valid_0's rmse: 2.72129
[349]	valid_0's rmse: 2.72178
[350]	valid_0's rmse: 2.72269
[351]	valid_0's rmse: 2.72313
[352]	valid_0's rmse: 2.72365
[353]	valid_0's rmse: 2.72269
[354]	valid_0's rmse: 2.72301
[355]	valid_0's rmse: 2.72373
[356]	valid_0's rmse: 2.72395
[357]	valid_0's rmse: 2.72371
[358]	valid_0's rmse: 2.72403
[359]	valid_

[651]	valid_0's rmse: 2.76539
[652]	valid_0's rmse: 2.76529
[653]	valid_0's rmse: 2.7651
[654]	valid_0's rmse: 2.76516
[655]	valid_0's rmse: 2.76562
[656]	valid_0's rmse: 2.76556
[657]	valid_0's rmse: 2.76559
[658]	valid_0's rmse: 2.76565
[659]	valid_0's rmse: 2.76624
[660]	valid_0's rmse: 2.76685
[661]	valid_0's rmse: 2.76706
[662]	valid_0's rmse: 2.76675
[663]	valid_0's rmse: 2.76712
[664]	valid_0's rmse: 2.76715
[665]	valid_0's rmse: 2.76664
[666]	valid_0's rmse: 2.76634
[667]	valid_0's rmse: 2.76674
[668]	valid_0's rmse: 2.76701
[669]	valid_0's rmse: 2.76697
[670]	valid_0's rmse: 2.76685
[671]	valid_0's rmse: 2.76723
[672]	valid_0's rmse: 2.76727
[673]	valid_0's rmse: 2.76736
[674]	valid_0's rmse: 2.76761
[675]	valid_0's rmse: 2.76723
[676]	valid_0's rmse: 2.76739
[677]	valid_0's rmse: 2.76726
[678]	valid_0's rmse: 2.76759
[679]	valid_0's rmse: 2.76802
[680]	valid_0's rmse: 2.768
[681]	valid_0's rmse: 2.76793
[682]	valid_0's rmse: 2.76823
[683]	valid_0's rmse: 2.76831
[684]	valid_0

[969]	valid_0's rmse: 2.78868
[970]	valid_0's rmse: 2.78862
[971]	valid_0's rmse: 2.78899
[972]	valid_0's rmse: 2.7893
[973]	valid_0's rmse: 2.78926
[974]	valid_0's rmse: 2.78944
[975]	valid_0's rmse: 2.78925
[976]	valid_0's rmse: 2.78909
[977]	valid_0's rmse: 2.78891
[978]	valid_0's rmse: 2.78906
[979]	valid_0's rmse: 2.78902
[980]	valid_0's rmse: 2.78927
[981]	valid_0's rmse: 2.78942
[982]	valid_0's rmse: 2.78968
[983]	valid_0's rmse: 2.7897
[984]	valid_0's rmse: 2.78993
[985]	valid_0's rmse: 2.78997
[986]	valid_0's rmse: 2.78972
[987]	valid_0's rmse: 2.78984
[988]	valid_0's rmse: 2.79016
[989]	valid_0's rmse: 2.79029
[990]	valid_0's rmse: 2.79026
[991]	valid_0's rmse: 2.79043
[992]	valid_0's rmse: 2.79035
[993]	valid_0's rmse: 2.79045
[994]	valid_0's rmse: 2.79041
[995]	valid_0's rmse: 2.79056
[996]	valid_0's rmse: 2.79075
[997]	valid_0's rmse: 2.79108
[998]	valid_0's rmse: 2.79095
[999]	valid_0's rmse: 2.79099
[1000]	valid_0's rmse: 2.79084
[1001]	valid_0's rmse: 2.79039
[1002]	val

[1291]	valid_0's rmse: 2.79989
[1292]	valid_0's rmse: 2.79991
[1293]	valid_0's rmse: 2.80035
[1294]	valid_0's rmse: 2.80068
[1295]	valid_0's rmse: 2.80096
[1296]	valid_0's rmse: 2.80113
[1297]	valid_0's rmse: 2.80093
[1298]	valid_0's rmse: 2.80093
[1299]	valid_0's rmse: 2.80132
[1300]	valid_0's rmse: 2.8012
[1301]	valid_0's rmse: 2.80109
[1302]	valid_0's rmse: 2.80112
[1303]	valid_0's rmse: 2.80127
[1304]	valid_0's rmse: 2.80129
[1305]	valid_0's rmse: 2.80142
[1306]	valid_0's rmse: 2.80163
[1307]	valid_0's rmse: 2.80181
[1308]	valid_0's rmse: 2.80177
[1309]	valid_0's rmse: 2.80199
[1310]	valid_0's rmse: 2.80165
[1311]	valid_0's rmse: 2.80162
[1312]	valid_0's rmse: 2.80149
[1313]	valid_0's rmse: 2.80169
[1314]	valid_0's rmse: 2.80167
[1315]	valid_0's rmse: 2.80178
[1316]	valid_0's rmse: 2.80197
[1317]	valid_0's rmse: 2.80207
[1318]	valid_0's rmse: 2.80222
[1319]	valid_0's rmse: 2.80219
[1320]	valid_0's rmse: 2.80213
[1321]	valid_0's rmse: 2.80198
[1322]	valid_0's rmse: 2.80194
[1323]	va

[1631]	valid_0's rmse: 2.81185
[1632]	valid_0's rmse: 2.81162
[1633]	valid_0's rmse: 2.81162
[1634]	valid_0's rmse: 2.81169
[1635]	valid_0's rmse: 2.81174
[1636]	valid_0's rmse: 2.81199
[1637]	valid_0's rmse: 2.81207
[1638]	valid_0's rmse: 2.81207
[1639]	valid_0's rmse: 2.81219
[1640]	valid_0's rmse: 2.81211
[1641]	valid_0's rmse: 2.81203
[1642]	valid_0's rmse: 2.81212
[1643]	valid_0's rmse: 2.8123
[1644]	valid_0's rmse: 2.81224
[1645]	valid_0's rmse: 2.81218
[1646]	valid_0's rmse: 2.81222
[1647]	valid_0's rmse: 2.81217
[1648]	valid_0's rmse: 2.81219
[1649]	valid_0's rmse: 2.81225
[1650]	valid_0's rmse: 2.81236
[1651]	valid_0's rmse: 2.81238
[1652]	valid_0's rmse: 2.81245
[1653]	valid_0's rmse: 2.81256
[1654]	valid_0's rmse: 2.81262
[1655]	valid_0's rmse: 2.81273
[1656]	valid_0's rmse: 2.81282
[1657]	valid_0's rmse: 2.81299
[1658]	valid_0's rmse: 2.81306
[1659]	valid_0's rmse: 2.81336
[1660]	valid_0's rmse: 2.8133
[1661]	valid_0's rmse: 2.81335
[1662]	valid_0's rmse: 2.81317
[1663]	val

In [57]:
submission_lgbm

,seg_id,time_to_failure
0,0,3.569635
1,1,5.097076
2,2,5.550940
3,3,8.409831
4,4,6.650740
5,5,2.660450
6,6,7.996492
7,7,3.629713
8,8,4.652942
9,9,2.729601


<h1>Blend LGB and XGB</h1>

In [59]:
test_segs = X_test_scaled_corr_refcv.index
submission = pd.DataFrame(columns=['seg_id', 'time_to_failure'])
submission.seg_id = test_segs
submission['time_to_failure'] = (submission_xgb['time_to_failure'] + submission_lgbm['time_to_failure'])/2
submission.to_csv('submission.csv', index=False, line_terminator='\n')

In [60]:
submission

,seg_id,time_to_failure
0,0,3.280260
1,1,4.936298
2,2,5.596788
3,3,8.401530
4,4,7.114454
5,5,2.811625
6,6,8.566453
7,7,4.200832
8,8,4.541158
9,9,2.770912
